In [3]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [4]:
def get_trip_data():
    # build url
    url = f'https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/tour_dem_tttot?format=JSON&unit=NR&unit=NR&c_dest=EU27_2020_FOR&c_dest=EU27_2020_FOR&purpose=TOTAL&purpose=PER&purpose=PROF&purpose=TOTAL&purpose=PER&purpose=PROF&duration=N_GE1&duration=N1-3&duration=N_GE4&duration=N_GE1&duration=N1-3&duration=N_GE4&lang=en'
    url_text = requests.get(url)

    trip_dict = json.loads(url_text.text)

    return trip_dict

In [5]:
data = get_trip_data()
data

{'version': '2.0',
 'class': 'dataset',
 'label': 'Trips by duration, purpose and main destination',
 'source': 'ESTAT',
 'updated': '2025-02-27T23:00:00+0100',
 'value': {'2144': 31962,
  '1988': 1236012,
  '1989': 1897332,
  '1990': 4102510,
  '1991': 4675543,
  '1760': 1390863,
  '1761': 1680574,
  '1762': 3253301,
  '1763': 3678589,
  '1772': 37953,
  '1774': 102076,
  '1775': 193702,
  '2096': 1532374,
  '2097': 1259831,
  '2098': 2685401,
  '1904': 71908,
  '1905': 56478,
  '1906': 216308,
  '1907': 273609,
  '1784': 575544,
  '1785': 490169,
  '1786': 1301715,
  '1787': 1701345,
  '1808': 6360670,
  '1809': 8497900,
  '1810': 12972090,
  '1811': 14296783,
  '1796': 711631,
  '1797': 763901,
  '1798': 1143824,
  '1799': 1295918,
  '1748': 16914983,
  '1749': 21784674,
  '1750': 41682870,
  '1751': 49432386,
  '1820': 141893,
  '1821': 109548,
  '1822': 328393,
  '1823': 408300,
  '1844': 24894,
  '1845': 39806,
  '1846': 87513,
  '1847': 85435,
  '1856': 1060415,
  '1857': 126291

In [6]:
# improve modularity of function below
def transform_index(index, sizes):
    coords = []
    for size in reversed(sizes):
        coords.append(index % size)
        index //= size
    return list(reversed(coords))


In [7]:
def get_trip_df(trip_dict):
    # access dimensions & ids, size, category, label
    dimensions = trip_dict['dimension']
    dimension_ids = trip_dict['id']
    dimension_sizes = trip_dict['size']
    dimension_labels = {
        dim: dimensions[dim]['category']['label']
        for dim in dimension_ids
    }

    # set total_items to 1 so no multiply to 0
    total_items = 1
    for size in dimension_sizes:
        total_items *= size
    
    records = []
    for idx in range(total_items):
         # convert vals to strings
         key = str(idx)
         if key in trip_dict['value']:
             # transform datapoints into coordinates to access each oen
             coords = transform_index(idx, dimension_sizes)
             record = {}
             for i, dim in enumerate(dimension_ids):
                 label_keys = list(dimension_labels[dim].keys())
                 record[dim] = dimension_labels[dim][label_keys[coords[i]]]
             record['value'] = trip_dict['value'][key]
             records.append(record)
    df = pd.DataFrame(records)
    return df
    

In [8]:
trip_dict = get_trip_data()
trip_df = get_trip_df(trip_dict)

trip_df

,freq,c_dest,purpose,duration,unit,geo,time,value
0,Annual,EU27 countries (from 2020) except reporting co...,Total,1 night or over,Number,European Union - 27 countries (from 2020),2020,80397451
1,Annual,EU27 countries (from 2020) except reporting co...,Total,1 night or over,Number,European Union - 27 countries (from 2020),2021,109539505
2,Annual,EU27 countries (from 2020) except reporting co...,Total,1 night or over,Number,European Union - 27 countries (from 2020),2022,205080131
3,Annual,EU27 countries (from 2020) except reporting co...,Total,1 night or over,Number,European Union - 27 countries (from 2020),2023,228685148
4,Annual,EU27 countries (from 2020) except reporting co...,Total,1 night or over,Number,Euro area – 20 countries (from 2023),2020,65271156
...,...,...,...,...,...,...,...,...
1041,Annual,EU27 countries (from 2020) except reporting co...,"Professional, business",4 nights or over,Number,Sweden,2021,312962
1042,Annual,EU27 countries (from 2020) except reporting co...,"Professional, business",4 nights or over,Number,Sweden,2022,571680
1043,Annual,EU27 countries (from 2020) except reporting co...,"Professional, business",4 nights or over,Number,Norway,2021,160509
1044,Annual,EU27 countries (from 2020) except reporting co...,"Professional, business",4 nights or over,Number,Norway,2022,323550


In [18]:
trip_df.rename(columns = {'purpose': 'Purpose', 'duration': 'Duration', 'geo': 'Country', 'time': 'Year', 'value': 'Num_Trips'}, inplace = True)
trip_df_filtered = trip_df[['Purpose', 'Duration', 'Country', 'Year', 'Num_Trips']]
trip_df_filtered

,Purpose,Duration,Country,Year,Num_Trips
0,Total,1 night or over,European Union - 27 countries (from 2020),2020,80397451
1,Total,1 night or over,European Union - 27 countries (from 2020),2021,109539505
2,Total,1 night or over,European Union - 27 countries (from 2020),2022,205080131
3,Total,1 night or over,European Union - 27 countries (from 2020),2023,228685148
4,Total,1 night or over,Euro area – 20 countries (from 2023),2020,65271156
...,...,...,...,...,...
1041,"Professional, business",4 nights or over,Sweden,2021,312962
1042,"Professional, business",4 nights or over,Sweden,2022,571680
1043,"Professional, business",4 nights or over,Norway,2021,160509
1044,"Professional, business",4 nights or over,Norway,2022,323550
